In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('train.csv', sep='\t', index_col=0)
test = pd.read_csv('test.csv', sep='\t', index_col=0)

In [4]:
train = train.rename({'0': 'target'}, axis=1)
test = test.rename({'0': 'target'}, axis=1)

In [5]:
#indirectly dropping constant features
binary = []
categorical = []
others = []
for col in train.drop(['target'], 1).columns:
    len_values = train[col].value_counts().shape[0]
    if len_values == 2:
        binary.append(col)
    elif (len_values > 2) & (len_values <= 10):
        categorical.append(col)
    elif len_values > 10:
        others.append(col)

In [6]:
features = binary + categorical + others

In [7]:
#correlation matrix
train_corr = train.corr()

In [8]:
neg_pos_train_corr = pd.DataFrame(data=np.zeros((4, len(features))), columns=features, index=['neg', 'neg_idx', 'pos', 'pos_idx'])

In [9]:
#finding the biggest and the lowest (< 0) correlations for each feature
for feature in features:
    neg_pos_train_corr.loc['neg', feature] = train_corr.loc[feature].sort_values().drop([feature]).min()
    neg_pos_train_corr.loc['neg_idx', feature] = train_corr.loc[feature].sort_values().drop([feature]).idxmin()
    neg_pos_train_corr.loc['pos', feature] = train_corr.loc[feature].sort_values().drop([feature]).max()
    neg_pos_train_corr.loc['pos_idx', feature] = train_corr.loc[feature].sort_values().drop([feature]).idxmax()

In [10]:
#highest indirectly correlated features
high_neg_corr = neg_pos_train_corr.T[(neg_pos_train_corr.loc['neg'] > -0.99) & \
                                     (neg_pos_train_corr.loc['neg'] < -0.9)][['neg', 'neg_idx']]

In [11]:
#highest directly correlated
high_pos_corr = neg_pos_train_corr.T[(neg_pos_train_corr.loc['pos'] < 0.99) & \
                                     (neg_pos_train_corr.loc['pos'] > 0.9)][['pos', 'pos_idx']]

In [12]:
high_pos_corr.sort_values(by='pos', ascending=False)

,pos,pos_idx
122,0.989875,115
115,0.989875,122
298,0.986225,335
325,0.985693,319
319,0.985693,325
35,0.966677,146
146,0.966677,35
323,0.964712,335
326,0.96018,325
39,0.955762,148


In [13]:
high_neg_corr.sort_values(by='neg')

,neg,neg_idx
322,-0.965317,326
326,-0.965317,322
299,-0.962447,331
331,-0.962447,299
325,-0.911986,322


In [14]:
f_pos_pairs = []
for idx in high_pos_corr.index:
    f_pos_pairs.append(tuple(sorted((idx, high_pos_corr.loc[idx].pos_idx))))

In [15]:
f_pos_pairs = set(f_pos_pairs)

In [16]:
f_neg_pairs = []
for idx in high_neg_corr.index:
    f_neg_pairs.append(tuple(sorted((idx, high_neg_corr.loc[idx].neg_idx))))

In [17]:
f_neg_pairs = set(f_neg_pairs)

In [18]:
new_f = []

In [19]:
#creating new features according to high correlation between some pairs of features
for i, j in f_pos_pairs:
    new_col = i+'-'+j
    train[new_col] = np.where(train[i].apply(round) == train[j].apply(round), 0, 1)
    test[new_col] = np.where(test[i].apply(round) == test[j].apply(round), 0, 1)
    new_f.append(new_col)

In [20]:
for i, j in f_neg_pairs:
    new_col = i+'-'+j
    train[new_col] = np.where(train[i].apply(round) != train[j].apply(round), 0, 1)
    test[new_col] = np.where(test[i].apply(round) != test[j].apply(round), 0, 1)
    new_f.append(new_col)

In [119]:
n_features = features + new_f

In [22]:
from sklearn.ensemble import RandomForestClassifier
feature_filter = RandomForestClassifier(
    random_state=42,
    n_estimators=10,
    n_jobs=3
)
feature_filter.fit(train[n_features], train.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=3,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [23]:
f_importance = pd.Series(data=feature_filter.feature_importances_, index=n_features)

In [24]:
zero_importance = f_importance[f_importance == 0].index.values

In [25]:
#dropping redundant (importance = 0)
non_red_features = n_features.copy()
for elem in zero_importance:
    non_red_features.remove(elem)

In [26]:
#size of new features
print(len(features), len(n_features), len(non_red_features))

341 358 327


In [27]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

## LightGBM

In [28]:
import lightgbm as lgb

In [29]:
n_rounds = 2000
params_lgb = {
    'objective':'binary',
    
    #other
    'learning_rate': 0.01,
    'num_threads': 3,
    'metric': 'auc',
    'min_data_in_leaf': 50,
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'subsample_freq': 1,
    'max_depth': 15,
    'max_bin': 150
}


lgb_train = lgb.Dataset(train[n_features], label=train.target, free_raw_data=False)

In [30]:
%%time
result = lgb.cv(params_lgb, lgb_train, n_rounds, folds=cv.split(train[n_features], train.target), early_stopping_rounds=100, verbose_eval=1)

[1]	cv_agg's auc: 0.685657 + 0.00498493
[2]	cv_agg's auc: 0.701277 + 0.00469206
[3]	cv_agg's auc: 0.704411 + 0.0044588
[4]	cv_agg's auc: 0.707457 + 0.00550672
[5]	cv_agg's auc: 0.708221 + 0.0065106
[6]	cv_agg's auc: 0.710054 + 0.00532246
[7]	cv_agg's auc: 0.710052 + 0.00575465
[8]	cv_agg's auc: 0.710076 + 0.00695777
[9]	cv_agg's auc: 0.710927 + 0.00794231
[10]	cv_agg's auc: 0.711065 + 0.00749885
[11]	cv_agg's auc: 0.711197 + 0.00728081
[12]	cv_agg's auc: 0.71169 + 0.00708544
[13]	cv_agg's auc: 0.71238 + 0.00689822
[14]	cv_agg's auc: 0.712381 + 0.00728506
[15]	cv_agg's auc: 0.712384 + 0.00752457
[16]	cv_agg's auc: 0.712489 + 0.00754379
[17]	cv_agg's auc: 0.712729 + 0.00735778
[18]	cv_agg's auc: 0.713066 + 0.00737282
[19]	cv_agg's auc: 0.713577 + 0.00699121
[20]	cv_agg's auc: 0.713596 + 0.00706561
[21]	cv_agg's auc: 0.713751 + 0.00689965
[22]	cv_agg's auc: 0.713938 + 0.00657707
[23]	cv_agg's auc: 0.714541 + 0.00635994
[24]	cv_agg's auc: 0.714627 + 0.0064016
[25]	cv_agg's auc: 0.714526 + 

[201]	cv_agg's auc: 0.730815 + 0.00598052
[202]	cv_agg's auc: 0.730867 + 0.00601992
[203]	cv_agg's auc: 0.731005 + 0.00594153
[204]	cv_agg's auc: 0.731117 + 0.00597938
[205]	cv_agg's auc: 0.731184 + 0.00600263
[206]	cv_agg's auc: 0.731228 + 0.005955
[207]	cv_agg's auc: 0.731305 + 0.00592736
[208]	cv_agg's auc: 0.731368 + 0.00595069
[209]	cv_agg's auc: 0.731493 + 0.00598452
[210]	cv_agg's auc: 0.731543 + 0.00598887
[211]	cv_agg's auc: 0.731598 + 0.00604297
[212]	cv_agg's auc: 0.731641 + 0.00602511
[213]	cv_agg's auc: 0.731764 + 0.00605461
[214]	cv_agg's auc: 0.731799 + 0.00608322
[215]	cv_agg's auc: 0.731872 + 0.00612951
[216]	cv_agg's auc: 0.73196 + 0.00613567
[217]	cv_agg's auc: 0.732031 + 0.00611698
[218]	cv_agg's auc: 0.732109 + 0.00612557
[219]	cv_agg's auc: 0.732157 + 0.0060978
[220]	cv_agg's auc: 0.732209 + 0.00613483
[221]	cv_agg's auc: 0.732326 + 0.00610392
[222]	cv_agg's auc: 0.73237 + 0.00614011
[223]	cv_agg's auc: 0.73243 + 0.00613618
[224]	cv_agg's auc: 0.732518 + 0.0061857

[399]	cv_agg's auc: 0.741109 + 0.00667551
[400]	cv_agg's auc: 0.74113 + 0.00669389
[401]	cv_agg's auc: 0.741157 + 0.00669374
[402]	cv_agg's auc: 0.741175 + 0.00667655
[403]	cv_agg's auc: 0.741191 + 0.00666667
[404]	cv_agg's auc: 0.741243 + 0.00661589
[405]	cv_agg's auc: 0.741287 + 0.00662003
[406]	cv_agg's auc: 0.741323 + 0.00658647
[407]	cv_agg's auc: 0.741339 + 0.00658912
[408]	cv_agg's auc: 0.741344 + 0.00659256
[409]	cv_agg's auc: 0.741393 + 0.00661882
[410]	cv_agg's auc: 0.741455 + 0.00659915
[411]	cv_agg's auc: 0.741493 + 0.00656003
[412]	cv_agg's auc: 0.741499 + 0.00657985
[413]	cv_agg's auc: 0.741507 + 0.00654682
[414]	cv_agg's auc: 0.741539 + 0.00653162
[415]	cv_agg's auc: 0.741599 + 0.00649926
[416]	cv_agg's auc: 0.741614 + 0.00650035
[417]	cv_agg's auc: 0.74166 + 0.00654435
[418]	cv_agg's auc: 0.741716 + 0.00653567
[419]	cv_agg's auc: 0.741726 + 0.00658621
[420]	cv_agg's auc: 0.741774 + 0.00656167
[421]	cv_agg's auc: 0.741789 + 0.00658964
[422]	cv_agg's auc: 0.741847 + 0.006

[596]	cv_agg's auc: 0.744546 + 0.00689494
[597]	cv_agg's auc: 0.744536 + 0.00688435
[598]	cv_agg's auc: 0.744555 + 0.00685954
[599]	cv_agg's auc: 0.744574 + 0.0068556
[600]	cv_agg's auc: 0.744579 + 0.00682816
[601]	cv_agg's auc: 0.744617 + 0.00681773
[602]	cv_agg's auc: 0.744615 + 0.00681302
[603]	cv_agg's auc: 0.744588 + 0.00677623
[604]	cv_agg's auc: 0.74459 + 0.00679262
[605]	cv_agg's auc: 0.74461 + 0.00679199
[606]	cv_agg's auc: 0.744653 + 0.00676731
[607]	cv_agg's auc: 0.744659 + 0.00678404
[608]	cv_agg's auc: 0.744657 + 0.00678842
[609]	cv_agg's auc: 0.744669 + 0.00675191
[610]	cv_agg's auc: 0.74467 + 0.00674473
[611]	cv_agg's auc: 0.744731 + 0.00675408
[612]	cv_agg's auc: 0.744737 + 0.00677285
[613]	cv_agg's auc: 0.744721 + 0.00680561
[614]	cv_agg's auc: 0.74471 + 0.00679923
[615]	cv_agg's auc: 0.744726 + 0.00681648
[616]	cv_agg's auc: 0.744716 + 0.00680884
[617]	cv_agg's auc: 0.744706 + 0.00678604
[618]	cv_agg's auc: 0.744697 + 0.0067963
[619]	cv_agg's auc: 0.744692 + 0.0067701

[793]	cv_agg's auc: 0.746114 + 0.00681731
[794]	cv_agg's auc: 0.746086 + 0.00679669
[795]	cv_agg's auc: 0.746112 + 0.00679977
[796]	cv_agg's auc: 0.746124 + 0.00678855
[797]	cv_agg's auc: 0.746127 + 0.00680238
[798]	cv_agg's auc: 0.746121 + 0.0068158
[799]	cv_agg's auc: 0.74611 + 0.00680926
[800]	cv_agg's auc: 0.746135 + 0.00681142
[801]	cv_agg's auc: 0.746148 + 0.00682475
[802]	cv_agg's auc: 0.746144 + 0.00683899
[803]	cv_agg's auc: 0.746161 + 0.00684761
[804]	cv_agg's auc: 0.746159 + 0.00681403
[805]	cv_agg's auc: 0.746153 + 0.00683322
[806]	cv_agg's auc: 0.74616 + 0.00684168
[807]	cv_agg's auc: 0.746157 + 0.00683662
[808]	cv_agg's auc: 0.746154 + 0.00685467
[809]	cv_agg's auc: 0.746148 + 0.00682642
[810]	cv_agg's auc: 0.746139 + 0.00682396
[811]	cv_agg's auc: 0.74614 + 0.00682453
[812]	cv_agg's auc: 0.746165 + 0.00681793
[813]	cv_agg's auc: 0.746181 + 0.00681535
[814]	cv_agg's auc: 0.746195 + 0.0067977
[815]	cv_agg's auc: 0.746191 + 0.00678642
[816]	cv_agg's auc: 0.746192 + 0.006771

[990]	cv_agg's auc: 0.746769 + 0.0066955
[991]	cv_agg's auc: 0.746759 + 0.00666414
[992]	cv_agg's auc: 0.746747 + 0.00667089
[993]	cv_agg's auc: 0.746758 + 0.00669315
[994]	cv_agg's auc: 0.746779 + 0.00669879
[995]	cv_agg's auc: 0.746796 + 0.00671296
[996]	cv_agg's auc: 0.746801 + 0.0067331
[997]	cv_agg's auc: 0.746802 + 0.00671862
[998]	cv_agg's auc: 0.746823 + 0.00671428
[999]	cv_agg's auc: 0.746818 + 0.00672914
[1000]	cv_agg's auc: 0.746805 + 0.00670369
[1001]	cv_agg's auc: 0.746842 + 0.00669699
[1002]	cv_agg's auc: 0.746824 + 0.00669326
[1003]	cv_agg's auc: 0.74682 + 0.00666716
[1004]	cv_agg's auc: 0.746802 + 0.00667451
[1005]	cv_agg's auc: 0.746803 + 0.00666767
[1006]	cv_agg's auc: 0.746814 + 0.00664573
[1007]	cv_agg's auc: 0.746823 + 0.00663582
[1008]	cv_agg's auc: 0.746809 + 0.00664858
[1009]	cv_agg's auc: 0.746833 + 0.0066415
[1010]	cv_agg's auc: 0.746823 + 0.00662674
[1011]	cv_agg's auc: 0.746834 + 0.00664719
[1012]	cv_agg's auc: 0.74685 + 0.00666513
[1013]	cv_agg's auc: 0.746

In [31]:
from sklearn.metrics import roc_auc_score
def check_train_score(p, lgb_data, data, target, kf, num_rounds):
    roc_auc = []
    for train, val in kf.split(data,target):
        temp_lgb_train = lgb_data.subset(train)
        temp_lgb_val = lgb_data.subset(val)
        temp_model = lgb.train(p, temp_lgb_train, num_rounds, verbose_eval=num_rounds)
        roc_auc.append([roc_auc_score(target[train],temp_model.predict(data.loc[train])), roc_auc_score(target[val],temp_model.predict(data.loc[val]))])
    return np.mean(roc_auc, axis=0)

check_train_score(params_lgb, lgb_train, train[n_features], train.target, cv, 1060)

array([0.89730765, 0.74704488])

## XGBoost

In [34]:
import xgboost as xgb

In [35]:
params_xgb = {
    'objective': 'binary:logistic',
    'eta': 0.01,
    'silent': 1,
    'nthread': 3,
    'random_seed': 1,
    'eval_metric': 'auc',
    
    #new parameters
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'min_child_weight': 3, 
    'tree_method': 'hist',
    'max_bin': 150,
    'grow_policy': 'lossguide',
    'max_leaves': 17
}

xgb_train = xgb.DMatrix(train[n_features], train.target, feature_names=n_features)

In [36]:
results = xgb.cv(
    params_xgb, 
    xgb_train, 
    num_boost_round=2000,
    early_stopping_rounds=100,
    folds=cv, 
    verbose_eval=1
)

[0]	train-auc:0.675051+0.00286909	test-auc:0.668885+0.0107632
[1]	train-auc:0.692399+0.00543427	test-auc:0.680462+0.00678081
[2]	train-auc:0.698981+0.00324977	test-auc:0.688375+0.00499132
[3]	train-auc:0.703655+0.00378569	test-auc:0.693051+0.00385245
[4]	train-auc:0.704284+0.00412509	test-auc:0.692462+0.00356324
[5]	train-auc:0.707107+0.00310948	test-auc:0.695014+0.00522209
[6]	train-auc:0.709163+0.0039023	test-auc:0.696982+0.00467242
[7]	train-auc:0.710706+0.00305499	test-auc:0.698865+0.00491644
[8]	train-auc:0.712382+0.00241771	test-auc:0.699514+0.00493442
[9]	train-auc:0.712509+0.00253745	test-auc:0.699525+0.00490704
[10]	train-auc:0.713668+0.00349947	test-auc:0.699865+0.00449749
[11]	train-auc:0.713171+0.00377504	test-auc:0.698685+0.00406907
[12]	train-auc:0.714689+0.00308987	test-auc:0.700399+0.00424974
[13]	train-auc:0.714432+0.00323569	test-auc:0.70012+0.00361296
[14]	train-auc:0.714607+0.0029182	test-auc:0.70011+0.00342538
[15]	train-auc:0.715138+0.00363219	test-auc:0.700572+0.

[128]	train-auc:0.736427+0.000442039	test-auc:0.715872+0.00589008
[129]	train-auc:0.736504+0.000407883	test-auc:0.715849+0.00585468
[130]	train-auc:0.736718+0.000449203	test-auc:0.715973+0.00580325
[131]	train-auc:0.736964+0.00033041	test-auc:0.716122+0.00577193
[132]	train-auc:0.73723+0.000336078	test-auc:0.716298+0.00575538
[133]	train-auc:0.737425+0.000400931	test-auc:0.716401+0.00569527
[134]	train-auc:0.737644+0.000440385	test-auc:0.716525+0.00569952
[135]	train-auc:0.737871+0.000348644	test-auc:0.716588+0.00566778
[136]	train-auc:0.738042+0.000258308	test-auc:0.716604+0.0055879
[137]	train-auc:0.73818+0.000240956	test-auc:0.716707+0.00552322
[138]	train-auc:0.738334+0.000176545	test-auc:0.716766+0.00545349
[139]	train-auc:0.738585+0.00019794	test-auc:0.716882+0.00538588
[140]	train-auc:0.738895+0.000218459	test-auc:0.717065+0.00530095
[141]	train-auc:0.739053+0.000246764	test-auc:0.717124+0.00523872
[142]	train-auc:0.739278+0.000314555	test-auc:0.717234+0.00527728
[143]	train-auc

[254]	train-auc:0.759419+0.0012145	test-auc:0.726417+0.00221599
[255]	train-auc:0.759606+0.00121841	test-auc:0.726441+0.00220777
[256]	train-auc:0.759774+0.00119864	test-auc:0.726496+0.00224581
[257]	train-auc:0.75997+0.00121054	test-auc:0.726516+0.00220526
[258]	train-auc:0.760141+0.00117304	test-auc:0.726612+0.00225167
[259]	train-auc:0.760362+0.00114671	test-auc:0.726676+0.00227398
[260]	train-auc:0.760507+0.00115356	test-auc:0.726734+0.00227678
[261]	train-auc:0.760603+0.00112185	test-auc:0.726763+0.00230727
[262]	train-auc:0.760746+0.00112545	test-auc:0.726853+0.00230404
[263]	train-auc:0.760916+0.00111217	test-auc:0.726951+0.00228329
[264]	train-auc:0.761068+0.00111513	test-auc:0.726987+0.00228815
[265]	train-auc:0.761204+0.00115612	test-auc:0.727091+0.00223984
[266]	train-auc:0.761358+0.0011291	test-auc:0.727169+0.00220227
[267]	train-auc:0.761505+0.00116362	test-auc:0.72724+0.00217738
[268]	train-auc:0.761673+0.00113473	test-auc:0.727288+0.00218159
[269]	train-auc:0.761859+0.00

[381]	train-auc:0.778193+0.00119143	test-auc:0.73291+0.00101784
[382]	train-auc:0.778335+0.00114871	test-auc:0.732939+0.00102643
[383]	train-auc:0.778455+0.00113776	test-auc:0.732975+0.00102723
[384]	train-auc:0.778598+0.00111597	test-auc:0.73306+0.000980042
[385]	train-auc:0.778735+0.00111746	test-auc:0.733073+0.000971894
[386]	train-auc:0.778842+0.00111078	test-auc:0.733101+0.000946034
[387]	train-auc:0.778978+0.00110493	test-auc:0.733154+0.000932738
[388]	train-auc:0.779091+0.00111111	test-auc:0.733186+0.000951426
[389]	train-auc:0.779217+0.00110091	test-auc:0.733211+0.000929468
[390]	train-auc:0.779324+0.00109916	test-auc:0.733221+0.000939528
[391]	train-auc:0.779454+0.00107371	test-auc:0.733254+0.000911286
[392]	train-auc:0.779598+0.00108885	test-auc:0.733247+0.000925792
[393]	train-auc:0.779731+0.00107438	test-auc:0.733269+0.000928807
[394]	train-auc:0.779855+0.00109983	test-auc:0.733309+0.000928186
[395]	train-auc:0.779978+0.00105422	test-auc:0.733349+0.000954306
[396]	train-auc

[506]	train-auc:0.793689+0.00107261	test-auc:0.736017+0.000590698
[507]	train-auc:0.793795+0.00107755	test-auc:0.736023+0.000591922
[508]	train-auc:0.793878+0.00105832	test-auc:0.736066+0.000624841
[509]	train-auc:0.794021+0.00105464	test-auc:0.736066+0.000617447
[510]	train-auc:0.794116+0.00105658	test-auc:0.736097+0.000625975
[511]	train-auc:0.7942+0.00108207	test-auc:0.736136+0.000654405
[512]	train-auc:0.794305+0.00107002	test-auc:0.736152+0.000641107
[513]	train-auc:0.794427+0.00107659	test-auc:0.736158+0.000639847
[514]	train-auc:0.794568+0.00106632	test-auc:0.736207+0.000634423
[515]	train-auc:0.79469+0.00109513	test-auc:0.736202+0.000636096
[516]	train-auc:0.794844+0.00108728	test-auc:0.736243+0.000627523
[517]	train-auc:0.794952+0.00109698	test-auc:0.736265+0.000641173
[518]	train-auc:0.795062+0.00109892	test-auc:0.736277+0.000618509
[519]	train-auc:0.795169+0.0010924	test-auc:0.73628+0.000653367
[520]	train-auc:0.795284+0.00109309	test-auc:0.736293+0.000633871
[521]	train-auc

...

[1012]	train-auc:0.840922+0.00141123	test-auc:0.740888+0.00145742
[1013]	train-auc:0.840993+0.00142048	test-auc:0.740895+0.00145031
[1014]	train-auc:0.841069+0.00141923	test-auc:0.740885+0.0014731
[1015]	train-auc:0.841145+0.00141104	test-auc:0.74089+0.00148658
[1016]	train-auc:0.841239+0.00141913	test-auc:0.740892+0.00150321
[1017]	train-auc:0.841293+0.00143173	test-auc:0.740902+0.00150084
[1018]	train-auc:0.84136+0.00143697	test-auc:0.740908+0.00150842
[1019]	train-auc:0.84143+0.0014521	test-auc:0.740898+0.00151171
[1020]	train-auc:0.841533+0.00146525	test-auc:0.740922+0.0014825
[1021]	train-auc:0.841613+0.00146267	test-auc:0.740934+0.00147152
[1022]	train-auc:0.841712+0.0014613	test-auc:0.740949+0.0014949
[1023]	train-auc:0.841776+0.00147564	test-auc:0.740961+0.00151216
[1024]	train-auc:0.841853+0.00147359	test-auc:0.74097+0.00150141
[1025]	train-auc:0.841956+0.00145141	test-auc:0.740955+0.00152054
[1026]	train-auc:0.842052+0.0014605	test-auc:0.740938+0.00152522
[1027]	train-auc:0.8

[1137]	train-auc:0.850307+0.00143105	test-auc:0.741316+0.00150265
[1138]	train-auc:0.850383+0.0014409	test-auc:0.741283+0.00150412
[1139]	train-auc:0.85044+0.00144849	test-auc:0.741271+0.0014968
[1140]	train-auc:0.85052+0.00144471	test-auc:0.741293+0.00148533
[1141]	train-auc:0.850577+0.0014288	test-auc:0.741302+0.00145842
[1142]	train-auc:0.850641+0.00141558	test-auc:0.741301+0.00146471
[1143]	train-auc:0.850725+0.00139213	test-auc:0.741285+0.00149352
[1144]	train-auc:0.850774+0.00139373	test-auc:0.741272+0.0015184
[1145]	train-auc:0.850822+0.00139034	test-auc:0.741279+0.00151039
[1146]	train-auc:0.85088+0.00135773	test-auc:0.741292+0.00148637
[1147]	train-auc:0.850947+0.00137942	test-auc:0.741286+0.00145238
[1148]	train-auc:0.851025+0.00138942	test-auc:0.741288+0.00144989
[1149]	train-auc:0.851094+0.00138108	test-auc:0.741301+0.00142297
[1150]	train-auc:0.851176+0.00138665	test-auc:0.741304+0.00143794
[1151]	train-auc:0.851253+0.00139279	test-auc:0.741309+0.00144159
[1152]	train-auc:

[1263]	train-auc:0.859172+0.00127569	test-auc:0.741697+0.00166193
[1264]	train-auc:0.859221+0.00127666	test-auc:0.741704+0.00165074
[1265]	train-auc:0.85927+0.00127954	test-auc:0.7417+0.00164982
[1266]	train-auc:0.859337+0.0012864	test-auc:0.741672+0.00167091
[1267]	train-auc:0.859411+0.00127777	test-auc:0.741674+0.00166539
[1268]	train-auc:0.859486+0.00126782	test-auc:0.741687+0.00165618
[1269]	train-auc:0.859553+0.00127916	test-auc:0.741695+0.00166242
[1270]	train-auc:0.859626+0.0012773	test-auc:0.741717+0.00166041
[1271]	train-auc:0.8597+0.00130302	test-auc:0.741712+0.00166386
[1272]	train-auc:0.859766+0.00130487	test-auc:0.741728+0.00165622
[1273]	train-auc:0.859829+0.00129931	test-auc:0.741739+0.00164661
[1274]	train-auc:0.859887+0.00131173	test-auc:0.741765+0.00165085
[1275]	train-auc:0.859942+0.00129431	test-auc:0.74176+0.00166411
[1276]	train-auc:0.860001+0.00129759	test-auc:0.741736+0.00165747
[1277]	train-auc:0.860059+0.0012862	test-auc:0.741719+0.00166694
[1278]	train-auc:0.

[1389]	train-auc:0.867021+0.00126572	test-auc:0.741968+0.00157168
[1390]	train-auc:0.867103+0.00127033	test-auc:0.741955+0.00156479
[1391]	train-auc:0.867141+0.00126382	test-auc:0.741974+0.0015643
[1392]	train-auc:0.867201+0.00128893	test-auc:0.741972+0.00157836
[1393]	train-auc:0.867247+0.00128822	test-auc:0.741981+0.00158165
[1394]	train-auc:0.867315+0.00127289	test-auc:0.742002+0.00157266
[1395]	train-auc:0.867373+0.00125999	test-auc:0.742003+0.00157186
[1396]	train-auc:0.867439+0.0012329	test-auc:0.742015+0.00158214
[1397]	train-auc:0.867499+0.00124188	test-auc:0.742012+0.00159396
[1398]	train-auc:0.867563+0.00125156	test-auc:0.741995+0.00159996
[1399]	train-auc:0.867626+0.00125516	test-auc:0.742003+0.00161315
[1400]	train-auc:0.867679+0.00125447	test-auc:0.742009+0.0016276
[1401]	train-auc:0.867727+0.00126275	test-auc:0.742+0.00163888
[1402]	train-auc:0.867797+0.00126912	test-auc:0.741995+0.00163517
[1403]	train-auc:0.867866+0.00125915	test-auc:0.742008+0.0016464
[1404]	train-auc:

[1514]	train-auc:0.874283+0.00118362	test-auc:0.741938+0.0017087
[1515]	train-auc:0.874329+0.00116185	test-auc:0.741934+0.00169416
[1516]	train-auc:0.874385+0.0011527	test-auc:0.741941+0.00169139
[1517]	train-auc:0.874458+0.00115396	test-auc:0.741937+0.00169212
[1518]	train-auc:0.87451+0.00116315	test-auc:0.741922+0.00169672
[1519]	train-auc:0.874569+0.00115621	test-auc:0.741927+0.00169783
[1520]	train-auc:0.874638+0.0011631	test-auc:0.74192+0.0017145
[1521]	train-auc:0.874699+0.00117664	test-auc:0.74192+0.00171761
[1522]	train-auc:0.874765+0.00114986	test-auc:0.741921+0.00172825
[1523]	train-auc:0.874819+0.00115013	test-auc:0.741941+0.00172839
[1524]	train-auc:0.874871+0.001155	test-auc:0.741957+0.00173181
[1525]	train-auc:0.874925+0.00115864	test-auc:0.74197+0.0017336
[1526]	train-auc:0.874975+0.00116067	test-auc:0.741976+0.00173439
[1527]	train-auc:0.87503+0.00117943	test-auc:0.741985+0.00173979
[1528]	train-auc:0.875096+0.00117126	test-auc:0.74197+0.00175881
[1529]	train-auc:0.8751

## Catboost

In [37]:
import catboost as ctb

In [45]:
X_train = train[n_features]
y_train = train.target
X_test = test[n_features]

In [46]:
ctb_train = ctb.Pool(X_train.values, y_train.values)

In [99]:
ctb_model = ctb.CatBoostClassifier(
    random_seed=42,
    eval_metric='AUC',
    rsm=0.7,
    l2_leaf_reg = 5,
    one_hot_max_size = 20,
    od_wait=10,
    od_type='Iter',
    eta=0.008,
    subsample=0.8,
    iterations=1450
)

In [100]:
from sklearn.metrics import roc_auc_score
scores = []
for train_idx, cv_idx in cv.split(X_train, y_train):
    temp_ctb_train = ctb.Pool(X_train.loc[train_idx].values, y_train.loc[train_idx].values)
    temp_ctb_val_X = ctb.Pool(X_train.loc[cv_idx, n_features].values)
    temp_ctb_val_y = y_train.loc[cv_idx].values
    ctb_model.fit(temp_ctb_train, verbose=True)
    prediction = ctb_model.predict_proba(temp_ctb_val_X)[:, 1]
    scores.append(roc_auc_score(temp_ctb_val_y, prediction))

Borders generated
0:	learn 0.6300645468passed: 0.405 sec	total: 4.43s	remaining: 1h 47m 4s
1:	learn 0.6529905538passed: 0.432 sec	total: 4.87s	remaining: 58m 43s
2:	learn 0.6581825904passed: 0.364 sec	total: 5.23s	remaining: 42m 3s
3:	learn 0.6661366442passed: 0.467 sec	total: 5.7s	remaining: 34m 20s
4:	learn 0.672852491passed: 0.393 sec	total: 6.09s	remaining: 29m 21s
5:	learn 0.6749300698passed: 0.388 sec	total: 6.48s	remaining: 26m
6:	learn 0.6757491024passed: 0.403 sec	total: 6.89s	remaining: 23m 39s
7:	learn 0.6773074258passed: 0.369 sec	total: 7.25s	remaining: 21m 47s
8:	learn 0.6763241753passed: 0.4 sec	total: 7.66s	remaining: 20m 25s
9:	learn 0.6772142568passed: 0.359 sec	total: 8.02s	remaining: 19m 14s
10:	learn 0.678373492passed: 0.389 sec	total: 8.4s	remaining: 18m 19s
11:	learn 0.6817522083passed: 0.391 sec	total: 8.8s	remaining: 17m 34s
12:	learn 0.6838839385passed: 0.392 sec	total: 9.19s	remaining: 16m 55s
13:	learn 0.6846531748passed: 0.463 sec	total: 9.65s	remaining: 16

115:	learn 0.7348628235passed: 0.459 sec	total: 51.1s	remaining: 9m 48s
116:	learn 0.7351856747passed: 0.439 sec	total: 51.6s	remaining: 9m 47s
117:	learn 0.7353754486passed: 0.409 sec	total: 52s	remaining: 9m 46s
118:	learn 0.7355766101passed: 0.438 sec	total: 52.4s	remaining: 9m 46s
119:	learn 0.7358428566passed: 0.432 sec	total: 52.9s	remaining: 9m 45s
120:	learn 0.7361440127passed: 0.393 sec	total: 53.3s	remaining: 9m 44s
121:	learn 0.7362860875passed: 0.369 sec	total: 53.6s	remaining: 9m 43s
122:	learn 0.7363331316passed: 0.511 sec	total: 54.1s	remaining: 9m 44s
123:	learn 0.7365727707passed: 0.427 sec	total: 54.6s	remaining: 9m 43s
124:	learn 0.7368464404passed: 0.492 sec	total: 55.1s	remaining: 9m 43s
125:	learn 0.7370795009passed: 0.557 sec	total: 55.6s	remaining: 9m 44s
126:	learn 0.7374209675passed: 0.486 sec	total: 56.1s	remaining: 9m 44s
127:	learn 0.7375746998passed: 0.527 sec	total: 56.6s	remaining: 9m 44s
128:	learn 0.7376850249passed: 0.445 sec	total: 57.1s	remaining: 9

229:	learn 0.756846442passed: 0.36 sec	total: 1m 37s	remaining: 8m 38s
230:	learn 0.7570012028passed: 0.356 sec	total: 1m 38s	remaining: 8m 37s
231:	learn 0.7572101322passed: 0.35 sec	total: 1m 38s	remaining: 8m 36s
232:	learn 0.7573541949passed: 0.358 sec	total: 1m 38s	remaining: 8m 35s
233:	learn 0.7575635379passed: 0.337 sec	total: 1m 39s	remaining: 8m 34s
234:	learn 0.7578055499passed: 0.345 sec	total: 1m 39s	remaining: 8m 34s
235:	learn 0.7579470731passed: 0.313 sec	total: 1m 39s	remaining: 8m 33s
236:	learn 0.7580452522passed: 0.324 sec	total: 1m 40s	remaining: 8m 32s
237:	learn 0.7582411393passed: 0.413 sec	total: 1m 40s	remaining: 8m 31s
238:	learn 0.7584733495passed: 0.533 sec	total: 1m 41s	remaining: 8m 31s
239:	learn 0.7586766138passed: 0.377 sec	total: 1m 41s	remaining: 8m 31s
240:	learn 0.7588760286passed: 0.347 sec	total: 1m 41s	remaining: 8m 30s
241:	learn 0.7589860664passed: 0.323 sec	total: 1m 42s	remaining: 8m 29s
242:	learn 0.7590519903passed: 0.4 sec	total: 1m 42s	r

343:	learn 0.7741703388passed: 0.362 sec	total: 2m 23s	remaining: 7m 41s
344:	learn 0.7743671337passed: 0.372 sec	total: 2m 23s	remaining: 7m 40s
345:	learn 0.7745714552passed: 0.442 sec	total: 2m 24s	remaining: 7m 40s
346:	learn 0.7747533118passed: 0.389 sec	total: 2m 24s	remaining: 7m 39s
347:	learn 0.7748415626passed: 0.378 sec	total: 2m 24s	remaining: 7m 39s
348:	learn 0.7750240972passed: 0.409 sec	total: 2m 25s	remaining: 7m 38s
349:	learn 0.7752243279passed: 0.415 sec	total: 2m 25s	remaining: 7m 38s
350:	learn 0.7753402836passed: 0.398 sec	total: 2m 26s	remaining: 7m 37s
351:	learn 0.7755301149passed: 0.399 sec	total: 2m 26s	remaining: 7m 37s
352:	learn 0.7756462085passed: 0.428 sec	total: 2m 27s	remaining: 7m 36s
353:	learn 0.7758945635passed: 0.372 sec	total: 2m 27s	remaining: 7m 36s
354:	learn 0.7760198269passed: 0.332 sec	total: 2m 27s	remaining: 7m 35s
355:	learn 0.7762008447passed: 0.382 sec	total: 2m 28s	remaining: 7m 35s
356:	learn 0.776247004passed: 0.347 sec	total: 2m 2

457:	learn 0.7934230633passed: 0.339 sec	total: 3m 5s	remaining: 6m 42s
458:	learn 0.7935945321passed: 0.347 sec	total: 3m 6s	remaining: 6m 41s
459:	learn 0.7936689018passed: 0.315 sec	total: 3m 6s	remaining: 6m 41s
460:	learn 0.7937584742passed: 0.297 sec	total: 3m 6s	remaining: 6m 40s
461:	learn 0.7938373599passed: 0.304 sec	total: 3m 6s	remaining: 6m 39s
462:	learn 0.7939809515passed: 0.31 sec	total: 3m 7s	remaining: 6m 39s
463:	learn 0.794086186passed: 0.339 sec	total: 3m 7s	remaining: 6m 38s
464:	learn 0.7941663703passed: 0.306 sec	total: 3m 7s	remaining: 6m 38s
465:	learn 0.794387618passed: 0.338 sec	total: 3m 8s	remaining: 6m 37s
466:	learn 0.7946241258passed: 0.3 sec	total: 3m 8s	remaining: 6m 36s
467:	learn 0.7947693031passed: 0.326 sec	total: 3m 8s	remaining: 6m 36s
468:	learn 0.7949196169passed: 0.323 sec	total: 3m 9s	remaining: 6m 35s
469:	learn 0.7950666557passed: 0.34 sec	total: 3m 9s	remaining: 6m 35s
470:	learn 0.7952151999passed: 0.304 sec	total: 3m 9s	remaining: 6m 34

...

789:	learn 0.8356455555passed: 0.348 sec	total: 4m 21s	remaining: 3m 38s
790:	learn 0.8357752985passed: 0.306 sec	total: 4m 21s	remaining: 3m 38s
791:	learn 0.8358848574passed: 0.291 sec	total: 4m 21s	remaining: 3m 37s
792:	learn 0.8360151059passed: 0.287 sec	total: 4m 22s	remaining: 3m 37s
793:	learn 0.8361384961passed: 0.319 sec	total: 4m 22s	remaining: 3m 36s
794:	learn 0.836259428passed: 0.309 sec	total: 4m 22s	remaining: 3m 36s
795:	learn 0.8363626227passed: 0.348 sec	total: 4m 23s	remaining: 3m 36s
796:	learn 0.8364945829passed: 0.318 sec	total: 4m 23s	remaining: 3m 35s
797:	learn 0.8366625227passed: 0.327 sec	total: 4m 23s	remaining: 3m 35s
798:	learn 0.8367714613passed: 0.305 sec	total: 4m 24s	remaining: 3m 35s
799:	learn 0.8368905437passed: 0.343 sec	total: 4m 24s	remaining: 3m 34s
800:	learn 0.8370255596passed: 0.352 sec	total: 4m 24s	remaining: 3m 34s
801:	learn 0.8371712591passed: 0.322 sec	total: 4m 25s	remaining: 3m 34s
802:	learn 0.8372780954passed: 0.325 sec	total: 4m 2

902:	learn 0.8468548146passed: 0.31 sec	total: 4m 57s	remaining: 3m
903:	learn 0.8469296001passed: 0.332 sec	total: 4m 58s	remaining: 3m
904:	learn 0.8470447536passed: 0.327 sec	total: 4m 58s	remaining: 2m 59s
905:	learn 0.8471183443passed: 0.293 sec	total: 4m 58s	remaining: 2m 59s
906:	learn 0.8472139686passed: 0.345 sec	total: 4m 59s	remaining: 2m 59s
907:	learn 0.8473735453passed: 0.313 sec	total: 4m 59s	remaining: 2m 58s
908:	learn 0.8474543274passed: 0.327 sec	total: 4m 59s	remaining: 2m 58s
909:	learn 0.8475912388passed: 0.328 sec	total: 5m	remaining: 2m 58s
910:	learn 0.847687334passed: 0.287 sec	total: 5m	remaining: 2m 57s
911:	learn 0.8477905977passed: 0.344 sec	total: 5m	remaining: 2m 57s
912:	learn 0.8478634417passed: 0.325 sec	total: 5m 1s	remaining: 2m 57s
913:	learn 0.8479871996passed: 0.346 sec	total: 5m 1s	remaining: 2m 56s
914:	learn 0.8481053054passed: 0.34 sec	total: 5m 1s	remaining: 2m 56s
915:	learn 0.8481955305passed: 0.332 sec	total: 5m 2s	remaining: 2m 56s
916:	

1128:	learn 0.8675098437passed: 0.166 sec	total: 5m 53s	remaining: 1m 40s
1129:	learn 0.8675821708passed: 0.164 sec	total: 5m 53s	remaining: 1m 40s
1130:	learn 0.86764162passed: 0.163 sec	total: 5m 53s	remaining: 1m 39s
1131:	learn 0.8676889956passed: 0.177 sec	total: 5m 54s	remaining: 1m 39s
1132:	learn 0.8678041606passed: 0.169 sec	total: 5m 54s	remaining: 1m 39s
1133:	learn 0.8679092623passed: 0.165 sec	total: 5m 54s	remaining: 1m 38s
1134:	learn 0.8679705841passed: 0.16 sec	total: 5m 54s	remaining: 1m 38s
1135:	learn 0.8680433477passed: 0.158 sec	total: 5m 54s	remaining: 1m 38s
1136:	learn 0.8681800178passed: 0.163 sec	total: 5m 54s	remaining: 1m 37s
1137:	learn 0.8682608688passed: 0.166 sec	total: 5m 55s	remaining: 1m 37s
1138:	learn 0.8683446148passed: 0.166 sec	total: 5m 55s	remaining: 1m 36s
1139:	learn 0.8684667069passed: 0.163 sec	total: 5m 55s	remaining: 1m 36s
1140:	learn 0.8685502345passed: 0.164 sec	total: 5m 55s	remaining: 1m 36s
1141:	learn 0.8685839397passed: 0.162 sec

1242:	learn 0.8765163554passed: 0.163 sec	total: 6m 12s	remaining: 1m 2s
1243:	learn 0.8765660745passed: 0.162 sec	total: 6m 12s	remaining: 1m 1s
1244:	learn 0.8766123933passed: 0.165 sec	total: 6m 12s	remaining: 1m 1s
1245:	learn 0.8767171618passed: 0.161 sec	total: 6m 13s	remaining: 1m 1s
1246:	learn 0.8768099601passed: 0.168 sec	total: 6m 13s	remaining: 1m
1247:	learn 0.8768539813passed: 0.174 sec	total: 6m 13s	remaining: 1m
1248:	learn 0.8768844354passed: 0.163 sec	total: 6m 13s	remaining: 1m
1249:	learn 0.876974086passed: 0.162 sec	total: 6m 13s	remaining: 59.8s
1250:	learn 0.8769763032passed: 0.165 sec	total: 6m 13s	remaining: 59.5s
1251:	learn 0.8770530875passed: 0.164 sec	total: 6m 14s	remaining: 59.2s
1252:	learn 0.877130676passed: 0.173 sec	total: 6m 14s	remaining: 58.8s
1253:	learn 0.8771825433passed: 0.164 sec	total: 6m 14s	remaining: 58.5s
1254:	learn 0.8772553299passed: 0.164 sec	total: 6m 14s	remaining: 58.2s
1255:	learn 0.877355779passed: 0.161 sec	total: 6m 14s	remaini

1356:	learn 0.8848991611passed: 0.166 sec	total: 6m 31s	remaining: 26.8s
1357:	learn 0.8849756352passed: 0.159 sec	total: 6m 31s	remaining: 26.5s
1358:	learn 0.8850150613passed: 0.167 sec	total: 6m 32s	remaining: 26.3s
1359:	learn 0.8850766243passed: 0.164 sec	total: 6m 32s	remaining: 26s
1360:	learn 0.8851814158passed: 0.161 sec	total: 6m 32s	remaining: 25.7s
1361:	learn 0.8853510329passed: 0.161 sec	total: 6m 32s	remaining: 25.4s
1362:	learn 0.8854260366passed: 0.165 sec	total: 6m 32s	remaining: 25.1s
1363:	learn 0.885485842passed: 0.166 sec	total: 6m 32s	remaining: 24.8s
1364:	learn 0.8855598463passed: 0.165 sec	total: 6m 33s	remaining: 24.5s
1365:	learn 0.8856160215passed: 0.162 sec	total: 6m 33s	remaining: 24.2s
1366:	learn 0.8857340125passed: 0.161 sec	total: 6m 33s	remaining: 23.9s
1367:	learn 0.8857990449passed: 0.16 sec	total: 6m 33s	remaining: 23.6s
1368:	learn 0.8858745885passed: 0.161 sec	total: 6m 33s	remaining: 23.3s
1369:	learn 0.8859730158passed: 0.165 sec	total: 6m 33s

In [101]:
np.mean(scores)

0.7463859437734237

In [50]:
lgb_predictions = []

for train_idx, cv_idx in cv.split(train[n_features], train.target):
    temp_lgb_train = lgb.Dataset(train.loc[train_idx, n_features], label=train.loc[train_idx, 'target'], free_raw_data=False)
    temp_lgb = lgb.train(params_lgb, temp_lgb_train, 1060)
    lgb_predictions.append([train.loc[cv_idx, 'target'].values, temp_lgb.predict(train.loc[cv_idx, n_features])])

In [52]:
xgb_predictions = []

for train_idx, cv_idx in cv.split(train[n_features], train.target):
    temp_xgb_train = xgb.DMatrix(train.loc[train_idx, n_features], train.loc[train_idx, 'target'], feature_names=n_features)
    temp_xgb = xgb.train(params_xgb, temp_xgb_train, 1459)
    temp_xgb_val = xgb.DMatrix(train.loc[cv_idx, n_features], feature_names=n_features)
    xgb_predictions.append([train.loc[cv_idx, 'target'].values, temp_xgb.predict(temp_xgb_val)])

In [103]:
ctb_predictions = []

for train_idx, cv_idx in cv.split(train[n_features], train.target):
    temp_ctb_train = ctb.Pool(train.loc[train_idx, n_features].values, train.loc[train_idx, 'target'].values)
    ctb_model.fit(temp_ctb_train)
    temp_ctb_val = ctb.Pool(train.loc[cv_idx, n_features].values)
    ctb_predictions.append([train.loc[cv_idx, 'target'].values, ctb_model.predict_proba(temp_ctb_val)[:, 1]])

In [54]:
np.mean([roc_auc_score(l_p[0], l_p[1]) for l_p in lgb_predictions])

0.7459479085449574

In [55]:
np.mean([roc_auc_score(x_p[0], x_p[1]) for x_p in xgb_predictions])

0.7452060964687534

In [104]:
np.mean([roc_auc_score(c_p[0], c_p[1]) for c_p in ctb_predictions])

0.7463859437734237

In [106]:
corr = []
for i in range(5):
    corr.append(np.corrcoef(lgb_predictions[i][1], ctb_predictions[i][1])[0, 1])
np.mean(corr)

0.9722577972742471

In [62]:
from scipy.stats import rankdata

## Simple ensemble

In [113]:
np.mean([roc_auc_score(c_p[0], rankdata(c_p[1]) + rankdata(l_p[1])) for c_p, l_p in zip(ctb_predictions, lgb_predictions)])

0.7477865529901953

In [128]:
ctb_model.fit(ctb_train)

In [129]:
ctb_test = ctb.Pool(test[n_features].values)
ctb_test_pred = ctb_model.predict_proba(ctb_test)[:, 1]

In [91]:
lgb_model = lgb.train(params_lgb, lgb_train, 1060)

In [92]:
lgb_test_pred = lgb_model.predict(test[n_features])

In [96]:
simple_ens_pred = rankdata(ctb_test_pred) + rankdata(lgb_test_pred)

In [97]:
predictions = pd.DataFrame(data=np.zeros((test.shape[0], 2)), columns=['_ID_', '_VAL_'])
predictions._ID_ = test.index.values
predictions._VAL_ = simple_ens_pred
predictions.to_csv('ens_.csv', index=False)

## Stacking

In [114]:
train['lgb'] = 0
train['ctb'] = 0 

In [120]:
n_features_stack = n_features + ['lgb', 'ctb']
for i, fold in enumerate(cv.split(train[n_features_stack], train.target)):
    train_idx, cv_idx = fold[0], fold[1]
    train.iloc[cv_idx, -2] = lgb_predictions[i][1]
    train.iloc[cv_idx, -1] = ctb_predictions[i][1]

In [121]:
train.head()

,target,1,2,3,4,5,6,7,8,9,...,146-35,158-20,148-39,110-117,298-335,322-325,322-326,299-331,lgb,ctb
0,1,1,0,0,0,0,0,0.090909,0,0,...,0,0,0,0,0,0,0,0,0.537140,0.566953
1,1,1,0,0,1,0,0,0.090909,0,0,...,0,0,0,0,0,0,0,0,0.171482,0.204402
2,0,1,0,0,1,0,0,0.090909,0,0,...,0,0,0,0,0,0,0,0,0.312348,0.271558
3,0,1,0,0,1,0,0,0.136364,0,0,...,0,0,0,0,0,0,0,0,0.245509,0.277764
4,0,1,0,0,1,0,0,0.136364,0,0,...,0,0,0,0,0,1,1,0,0.638262,0.642271


In [123]:
xgb_stack_train = xgb.DMatrix(train[n_features_stack], train.target, feature_names=n_features_stack)

In [126]:
nr_stacking = 500

params_stack = {
    'objective': 'binary:logistic',
    'eta': 0.01,
    'silent': 1,
    'nthread': 3,
    'random_seed': 17,
    'eval_metric': 'auc',
    
    #new parameters
    'subsample': 0.65,
    'colsample_bytree': 0.7,
    'min_child_samples': 300, 
    'num_leaves': 10
}

In [127]:
results = xgb.cv(
    params_stack, 
    xgb_stack_train, 
    num_boost_round=500,
    early_stopping_rounds=100,
    folds=cv, 
    verbose_eval=1
)

[0]	train-auc:0.749609+0.000652938	test-auc:0.730372+0.00113241
[1]	train-auc:0.759747+0.00075208	test-auc:0.738403+0.00262351
[2]	train-auc:0.764208+0.00108693	test-auc:0.739627+0.00101312
[3]	train-auc:0.766971+0.000625467	test-auc:0.741184+0.0012496
[4]	train-auc:0.769096+0.000384193	test-auc:0.74204+0.00153327
[5]	train-auc:0.770752+0.000635589	test-auc:0.742852+0.00102656
[6]	train-auc:0.771852+0.00074678	test-auc:0.743092+0.00112875
[7]	train-auc:0.772948+0.000821818	test-auc:0.743428+0.00111814
[8]	train-auc:0.773847+0.000774491	test-auc:0.743784+0.00128307
[9]	train-auc:0.774613+0.00053231	test-auc:0.744146+0.00117818
[10]	train-auc:0.774924+0.000512998	test-auc:0.744123+0.00136585
[11]	train-auc:0.775825+0.00040831	test-auc:0.74414+0.00174233
[12]	train-auc:0.77636+0.000234806	test-auc:0.744462+0.00168613
[13]	train-auc:0.776461+8.93396e-05	test-auc:0.744473+0.00167604
[14]	train-auc:0.776657+0.000245412	test-auc:0.744537+0.00146383
[15]	train-auc:0.776983+0.000391267	test-auc

[127]	train-auc:0.796412+0.000961951	test-auc:0.745891+0.000963127
[128]	train-auc:0.79654+0.000898784	test-auc:0.74586+0.000952984
[129]	train-auc:0.796663+0.000913622	test-auc:0.74586+0.000975572
[130]	train-auc:0.796793+0.000901835	test-auc:0.745877+0.000930936
[131]	train-auc:0.796875+0.000908011	test-auc:0.74587+0.000914714
[132]	train-auc:0.79705+0.000962245	test-auc:0.745873+0.00093855
[133]	train-auc:0.797165+0.000950278	test-auc:0.74587+0.000957984
[134]	train-auc:0.797346+0.000959966	test-auc:0.745835+0.000989963
[135]	train-auc:0.797504+0.000854166	test-auc:0.745867+0.000995759
[136]	train-auc:0.797644+0.000797853	test-auc:0.745864+0.00102729
[137]	train-auc:0.797754+0.000820286	test-auc:0.745901+0.00103426
[138]	train-auc:0.798003+0.000851918	test-auc:0.745895+0.00102941
[139]	train-auc:0.798269+0.000991917	test-auc:0.745906+0.00104661
[140]	train-auc:0.798422+0.00102246	test-auc:0.745919+0.00102357
[141]	train-auc:0.798565+0.00109571	test-auc:0.745928+0.00100292
[142]	trai

In [130]:
test['lgb'] = lgb_test_pred
test['ctb'] = ctb_test_pred

In [133]:
stacker = xgb.XGBClassifier(
    n_jobs=3,
    objective='binary:logistic',
    num_leaves=10,
    learning_rate=0.01,
    n_estimators=78,
    colsample_bytree=0.7,
    random_state=17,
    subsample=0.65,
    min_child_samples=300
)

stacker.fit(train[n_features_stack], train.target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_samples=300, min_child_weight=1,
       missing=None, n_estimators=78, n_jobs=3, nthread=None,
       num_leaves=10, objective='binary:logistic', random_state=17,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.65)

In [134]:
test_stack_pred = stacker.predict_proba(test[n_features_stack])[:, 1]

In [136]:
stack_pred_rank = rankdata(test_stack_pred)

In [137]:
predictions_stack = pd.DataFrame(data=np.zeros((test.shape[0], 2)), columns=['_ID_', '_VAL_'])
predictions_stack._ID_ = test.index.values
predictions_stack._VAL_ = stack_pred_rank
predictions_stack.to_csv('stack_gb.csv', index='_ID_', columns=['_VAL_'])